In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 9.0 MB/s 
     |████████████████████████████████| 86 kB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 47.6 MB/s 
     |████████████████████████████████| 596 kB 30.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn as nn
import pandas as pd

In [5]:
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")

In [6]:
def getFinBERTVal(sentence) :
    inputs = tokenizer(
                        sentence,
                        return_tensors='pt',
                        truncation=True,
                        max_length=128,
                        padding='max_length',
                        add_special_tokens=True
    )

    output = model(input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
    )

    m = nn.Softmax(dim=1)
    result = m(output[0])

    negative_val = float(result[0][0])
    neutral_val = float(result[0][1])
    positive_val = float(result[0][2])

    return negative_val, neutral_val, positive_val

In [8]:
# from google.colab import files
# uploaded = files.upload()

Saving cleaned_report.xlsx to cleaned_report.xlsx


In [13]:
filename = "./combined_dataset_cleaned.xlsx"
df = pd.read_excel(filename)
df[['negative', 'neutral', 'positive']] = pd.DataFrame([[0.0, 0.0, 0.0]], index=df.index)

logCnt = 100
plusCnt = 100
for i in range(len(df)) :
    if(i == logCnt) :
        print(i)
        logCnt += plusCnt
        
    negVal, neuVal, posVal = getFinBERTVal(df['title'][i])
    df['negative'][i] = negVal
    df['neutral'][i] = neuVal
    df['positive'][i] = posVal
print(df)

C:\Users\user\AppData\Local\Temp/ipykernel_9864/2921359054.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['negative'][i] = negVal
C:\Users\user\AppData\Local\Temp/ipykernel_9864/2921359054.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['neutral'][i] = neuVal
C:\Users\user\AppData\Local\Temp/ipykernel_9864/2921359054.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['positive'][i] = posVal


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
            date                                              title  negative  \
0     2021-06

In [14]:
print("Done!")

Done!


In [15]:
df.to_excel("./combined_dataset_cleaned_finBERT.xlsx", index = False)